## 0.1 Overview

This document is structured in three parts:
    
1) [**Importing Stuff**](#1): We define a rational approximation of the Ruperthedron and import the solution-tree.
2) [**Defining Functions**](#2): We implement a lot of functions, that are despribed in the paper. All of these will have rational inputs and outputs! In the paper, we denoted rational functions with \tilde. However, as here all functions are rational, **we omit this notion to make everything better readable.** (Except the rational approximations of sin and cos)
3) [**Verifying**](#3): Finally! We are ready to verify the solution-tree! This part is split in 3 parts:
   - [**3.1 General solution-tree integrety**](#3.1): We make sure, that the tree does not have any obvious faults. We check, that the root of the tree is what it should be, that the union of the regions of every child is indeed the region of their parent. This is essentially it.
   - [**3.2 Global Theorems**](#3.2): We check, that each claim of a leaf node being able to be resolved using the global theorems is true.
   - [**3.3 Local Theorems**](#3.3): We check, that each claim of a leaf node being able to be resolved using the global theorems is true.

## 0.2 The tree-structure

As said before, every node corresponds to a certain region in $\mathbb{R}^5$. It can either be directly shown that it does not contain a solution, or it is split into regions, such that each of them can then be verified (possibly by splitting them again). Eventually, every leaf node can be shown directly, using one of two theorems and given some specific parameters.

The solution-tree stores all those nodes and additional information. It is stored in a table, each row of which corresponding to a node of the tree, and the very first row of the tree corresponding to the root of the tree. In order to save memory, every entry of the table/every parameter is stored as an integer. To do so, some trickeries where used.

The information of every node, hence the columns of the tree are:
1) asd

# 1 Importing Stuff  <a id="1"></a>

We start by importing dependencies. These are just: 
1. Sage-packages
2. The constants **INTERVAL_DENOMINATOR** and **kappa**
3. [**The Nopert**](#1.1)
    -  The vertices of the Nopert, stored in a 90 x 3 matrix called **poly**
    -  We choose a rational approximations of the vertices of the Nopert, stored in a 90 x 3 matrix called **polyRat**, that is used throuout this script.
4. [**The Solution tree**](#1.2), stored as a dataframe called **TREE**

In [11]:
import pandas as pd
from sys import getsizeof
import time
INTERVAL_DENOMINATOR = 15360000
kappa=10^(-10)
NODETYPE_WEAK=1
NODETYPE_STRONG=2
NODETYPE_INNER_NODE=3
PATH="./sexypoly-solutionTree_try22012025.csv"
#PATH="~/Documents/sexypoly-solutionTree_try22012025.csv"

In [12]:
## unimportant code - just for timekeeping:

def remainingTime(i,n,startTime):
    if i==0:
        i=1
    averageIterationTime=(time.time()-startTime)/i
    remainingTime=floor((n-i)*averageIterationTime)

    hours = remainingTime // 3600
    minutes = (remainingTime % 3600) // 60
    seconds = remainingTime % 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"

## 1.1 Defining the (rational) Noperthedron <a id="1.1"></a>

First, we define the sexygon and fix its rational approximation

In [13]:
# the 3 points used to generate the Nopert and their norms:
C1=vector([152024884,0 ,210152163])/259375205
C2=vector([6632738028,6106948881,3980949609])/10^10
C3=vector([8193990033,5298215096,1230614493])/10^10

normC1_squared=C1[0]^2+C1[1]^2+C1[2]^2
normC2_squared=C2[0]^2+C2[1]^2+C2[2]^2
normC3_squared=C3[0]^2+C3[1]^2+C3[2]^2

print(f"The point C1 has a norm² of {normC1_squared}")
print(f"The point C2 has a norm² of {normC2_squared} = {normC2_squared+0.0}... <1")
print(f"The point C3 has a norm² of {normC3_squared} = {normC3_squared+0.0}... <1")
print("As claimed, their maximal norm is indeed 1.")

The point C1 has a norm² of 1
The point C2 has a norm² of 48567999086310866913/50000000000000000000 = 0.971359981726217... <1
The point C3 has a norm² of 48363483947383638677/50000000000000000000 = 0.967269678947673... <1
As claimed, their maximal norm is indeed 1.


Calculating the vertices of the Nopert and storing them as "poly":

In [14]:
# Rz(alpha) is the 3-dimensional rotation matrix around the z-axis as defined in the paper 
def Rz(alpha):
    return matrix([[cos(alpha),-sin(alpha),0],[sin(alpha),cos(alpha),0],[0,0,1]])

## We fill a 90 x 3 matrix with the vertices in the order described in the paper
poly= matrix(SR,90, 3) 
for i in range(3):
    for k in range(15):
        for l in range(2):
            if i==0:
                poly[k+15*i+45*l,:]=(-1)^l*Rz(2*pi*k/15)*C1
            if i==1:
                poly[k+15*i+45*l,:]=(-1)^l*Rz(2*pi*k/15)*C2
            if i==2:
                poly[k+15*i+45*l,:]=(-1)^l*Rz(2*pi*k/15)*C3
print("The first 5 points are")
pretty_print(poly[0:5,:])

The first 5 points are


[                                                                           152024884/259375205                                                                                              0                                                                            210152163/259375205]
[38006221/518750410*sqrt(5) + 38006221/259375205*sqrt(-3/2*sqrt(5) + 15/2) + 38006221/518750410                                                               152024884/259375205*sin(2/15*pi)                                                                            210152163/259375205]
[-38006221/518750410*sqrt(5) + 38006221/259375205*sqrt(3/2*sqrt(5) + 15/2) + 38006221/518750410                                                               152024884/259375205*sin(4/15*pi)                                                                            210152163/259375205]
[                                               38006221/259375205*sqrt(5) - 38006221/259375205                                                        38006221/259375205*sqrt(2*sqrt(5) + 10)                                                                            210152163/259375205]
[38006221/518750410*sqrt(5) - 38006221/259375205*sqrt(-3/2*sqrt(5) + 15/2) + 38006221/518750410                                                               152024884/259375205*sin(7/15*pi)                                                                            210152163/259375205]

Fixing the rational approximation of the Nopert. It is kappa close.

In [10]:
polyRat=matrix(QQ,90,3)
## check the dokuentation of "floor"

for row in range(poly.nrows()):
    for col in range(poly.ncols()):
        polyRat[row,col]=floor(poly[row,col]*10^16)/10^16

print("The first 5 rational points are")
pretty_print(polyRat[0:5,:])

The first 5 rational points are


[   183162366078901/312500000000000                                  0  4051122831883641/5000000000000000]
[5354468721358439/10000000000000000     74498845916753/312500000000000  4051122831883641/5000000000000000]
[ 1960952721223971/5000000000000000 4355717266359407/10000000000000000  4051122831883641/5000000000000000]
[  905604541572893/5000000000000000   557432837758007/1000000000000000  4051122831883641/5000000000000000]
[-612661780950237/10000000000000000 5829087471133637/10000000000000000  4051122831883641/5000000000000000]

In [ ]:
## verify that it is kappa close: AUCH DIE EUKLIDISCHE

## 1.2 Importing the solution-tree as **TREE** <a id="1.2"></a>

All nodes will be stored in a dataframe (?). Each node will store different parameters, that will be the columns of the table.

First we decide on which datatype to use for each column.

In [9]:
column_dtypes = {}

column_dtypes['ID']= 'Int32'
column_dtypes['nodetype']= 'Int8'
column_dtypes['nrChildren']= 'Int8'
column_dtypes['IDfirstChild']= 'Int32'
column_dtypes['split']= 'Int8'
column_dtypes['T1_min']= 'Int8'
column_dtypes['T1_max']= 'Int8'
column_dtypes['V1_min']= 'Int8'
column_dtypes['V1_max']= 'Int8'
column_dtypes['T2_min']= 'Int8'
column_dtypes['T2_max']= 'Int8'
column_dtypes['V2_min']= 'Int8'
column_dtypes['V2_max']= 'Int8'
column_dtypes['A_min']= 'Int32'
column_dtypes['A_max']= 'Int32'
column_dtypes['S_index']="Int8"
column_dtypes['P1_index']="Int8"
column_dtypes['P2_index']="Int8"
column_dtypes['P3_index']="Int8"
column_dtypes['Q1_index']="Int8"
column_dtypes['Q2_index']="Int8"
column_dtypes['Q3_index']="Int8"
column_dtypes['s_q']="Int8"
column_dtypes['wx_nominator']="Int64"
column_dtypes['wy_nominator']="Int64"
column_dtypes['w_denominator']="Int64"
column_dtypes['r']="Int16"
column_dtypes

{'ID': 'Int32',
 'nodetype': 'Int8',
 'nrChildren': 'Int8',
 'IDfirstChild': 'Int32',
 'split': 'Int8',
 'T1_min': 'Int8',
 'T1_max': 'Int8',
 'V1_min': 'Int8',
 'V1_max': 'Int8',
 'T2_min': 'Int8',
 'T2_max': 'Int8',
 'V2_min': 'Int8',
 'V2_max': 'Int8',
 'A_min': 'Int32',
 'A_max': 'Int32',
 'S_index': 'Int8',
 'P1_index': 'Int8',
 'P2_index': 'Int8',
 'P3_index': 'Int8',
 'Q1_index': 'Int8',
 'Q2_index': 'Int8',
 'Q3_index': 'Int8',
 's_q': 'Int8',
 'wx_nominator': 'Int64',
 'wy_nominator': 'Int64',
 'w_denominator': 'Int64',
 'r': 'Int16'}

Importing the solution tree as 'TREE'

In [10]:
chunksize=10000
chunkdata=[]
startTime=time.time()
i=0


for chunk in pd.read_csv(PATH, sep=';',  dtype=column_dtypes, chunksize=int(chunksize)):
    chunkdata.append(chunk)

    i+=1
    print(f"Processing: {i} of 1871; remaining: {remainingTime(i,1871,startTime)}", end='\r')
    
TREE=pd.concat(chunkdata,ignore_index=True)
del(chunkdata)

In [9]:
print(type(TREE))
TREE ## takes like 20 seconds

<class 'pandas.core.frame.DataFrame'>


,ID,nodetype,nrChildren,IDfirstChild,split,T1_min,T1_max,V1_min,V1_max,T2_min,...,P2_index,P3_index,Q1_index,Q2_index,Q3_index,r,s_q,wx_nominator,wy_nominator,w_denominator
0,1,3,4,2,1,0,6451200,0,48384000,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2,3,30,6,2,0,1612800,0,48384000,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,3,3,30,4693268,2,1612800,3225600,0,48384000,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,4,3,30,9459778,2,3225600,4838400,0,48384000,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,5,3,30,14086352,2,4838400,6451200,0,48384000,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18700440,18700441,1,<NA>,<NA>,<NA>,4838400,6451200,46771200,48384000,4838400,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3310140,-1481478251,1481481949
18700441,18700442,1,<NA>,<NA>,<NA>,4838400,6451200,46771200,48384000,4838400,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3310140,-1481478251,1481481949
18700442,18700443,1,<NA>,<NA>,<NA>,4838400,6451200,46771200,48384000,4838400,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3310140,-1481478251,1481481949
18700443,18700444,1,<NA>,<NA>,<NA>,4838400,6451200,46771200,48384000,4838400,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3310140,-1481478251,1481481949


# 2 Defining Functions  <a id="2"></a>

Here we define many of the functions outlined in the paper. All of them are rational functions. They are seperated in sections: 

1. [**Trigonometric functions**](#2.1): Rational approximations of sine and cosine
2. [**Defining Matrices**](#2.2): Many rotation/projection matrices
3. [**Miscellaneous functions**](#2.3): some clever stuff

## 2.1 Trigonometric functions <a id="2.1"></a>

In [11]:
def sinQ(x):
    assert isinstance(x, Rational) or isinstance(x, Integer), f"Give me rational inputs, please! You gave me {type(x)}"
    assert abs(x)<=4, f"x should be between -4 and 4 and you gave me {x}"
    
    x2 = x^2
    
    return x*(1-x2/(2*3)*
             (1-x2/(4*5)*
             (1-x2/(6*7)*
             (1-x2/(8*9)*
             (1-x2/(10*11)*
             (1-x2/(12*13)*
             (1-x2/(14*15)*
             (1-x2/(16*17)*
             (1-x2/(18*19)*
             (1-x2/(20*21)*
             (1-x2/(22*23)*
             (1-x2/(24*25)))))))))))))

def cosQ(x):
    assert isinstance(x, Rational) or isinstance(x, Integer), f"Give me rational inputs, please! You gave me {type(x)}"
    assert abs(x)<=4, f"x should be between -4 and 4 and you gave me {x}"
    
    x2 = x^2
  
    return (1-x2/(1*2)*
           (1-x2/(3*4)*
           (1-x2/(5*6)*
           (1-x2/(7*8)*
           (1-x2/(9*10)*
           (1-x2/(11*12)*
           (1-x2/(13*14)*
           (1-x2/(15*16)*
           (1-x2/(17*18)*
           (1-x2/(19*20)*
           (1-x2/(21*22)*
           (1-x2/(23*24)))))))))))))

**Example Code**: Let's approximate some trigonometic functions

In [12]:
print(f"sin(2)  = {sin(2).n(75)}...")
print(f"sinQ(2) = {sinQ(2).n(75)}... = {sinQ(2)}")
print(f"cos(2)  = {cos(2).n(75)}...")
print(f"cosQ(2) = {cosQ(2).n(75)}... = {cosQ(2)}")

sin(2)  = 0.9092974268256816953960...
sinQ(2) = 0.9092974268256816954083... = 176985682680149282/194640034667203125
cos(2)  = -0.4161468365471423869976...
cosQ(2) = -0.4161468365471423868320... = -61559114366058857/147926426347074375


## 2.2 Defining Matrices <a id="2.2"></a> 

In [24]:
def abs_matrix(M): ## every entry is replaced by its absolute value
    #if not isinstance(M, Matrix): #Check if it is a matrix
    #    raise TypeError("Input must be a SageMath Matrix")
    nrows = M.nrows()
    ncols = M.ncols()
    new_matrix = matrix(M.base_ring(), nrows, ncols) #Create an empty matrix with the same base ring
    for i in range(nrows):
        for j in range(ncols):
            new_matrix[i,j] = abs(M[i,j])
    return new_matrix

In [25]:
def X(theta,phi):
    assert isinstance(theta, Rational) or isinstance(theta, Integer), f"Give me rational inputs, please! You gave me {type(theta)}"
    assert isinstance(phi  , Rational) or isinstance(phi  , Integer), f"Give me rational inputs, please! You gave me {type(phi  )}"
    
    A=matrix(QQ,3,1)
    A[:,0]=matrix([[cosQ(theta)*sinQ(phi)],[sinQ(theta)*sinQ(phi)],[cosQ(phi)]])

    return A 

In [26]:
def M(theta,phi):
    assert isinstance(theta, Rational) or isinstance(theta, Integer), f"Give me rational inputs, please! You gave me {type(theta)}"
    assert isinstance(phi  , Rational) or isinstance(phi  , Integer), f"Give me rational inputs, please! You gave me {type(phi  )}"
    
    A=matrix(QQ,2,3)
    A[0,:]=matrix([-sinQ(theta),cosQ(theta),0])
    A[1,:]=matrix([-cosQ(theta)*cosQ(phi),-sinQ(theta)*cosQ(phi),sinQ(phi)])

    return A 

In [27]:
def R(alpha):
    assert isinstance(alpha, Rational) or isinstance(alpha, Integer), f"Give me rational inputs, please! You gave me {type(alpha)}"

    A=matrix(QQ,2,2)
    A[0,:]=matrix([cosQ(alpha),-sinQ(alpha)])
    A[1,:]=matrix([sinQ(alpha),cosQ(alpha)])

    return A

In [28]:
def M_theta_prime(theta,phi):
    assert isinstance(theta, Rational) or isinstance(theta, Integer), f"Give me rational inputs, please! You gave me {type(theta)}"
    assert isinstance(phi  , Rational) or isinstance(phi  , Integer), f"Give me rational inputs, please! You gave me {type(phi  )}"
    
    A=matrix(QQ,2,3)
    A[0,:]=matrix([-cosQ(theta),-sinQ(theta),0])
    A[1,:]=matrix([sinQ(theta)*cosQ(phi),-cosQ(theta)*cosQ(phi),0])

    return A

In [29]:
def M_phi_prime(theta,phi):
    assert isinstance(theta, Rational) or isinstance(theta, Integer), f"Give me rational inputs, please! You gave me {type(theta)}"
    assert isinstance(phi  , Rational) or isinstance(phi  , Integer), f"Give me rational inputs, please! You gave me {type(phi  )}"
    
    A=matrix(QQ,2,3)
    A[0,:]=matrix([0,0,0])
    A[1,:]=matrix([cosQ(theta)*sinQ(phi),sinQ(theta)*sinQ(phi),cosQ(phi)])

    return A

In [30]:
def R_alpha_prime(alpha):
    assert isinstance(alpha, Rational) or isinstance(alpha, Integer), f"Give me rational inputs, please! You gave me {type(alpha)}"

    A=matrix(QQ,2,2)
    A[0,:]=matrix([-sinQ(alpha),-cosQ(alpha)])
    A[1,:]=matrix([cosQ(alpha),-sinQ(alpha)])

    return A

## 2.3 Miscellaneous functions <a id="2.3"></a>

In [31]:
def ScalarProduct(vector1,vector2):
    ## returns the scalar product of two (vertical) vectors, aka n times 1 matrices
    
    assert vector1.ncols()==1, f"ScalarProduct wants integers or matrix as input"
    assert vector2.ncols()==1, f"ScalarProduct wants integers or matrix as input"
    assert vector1.nrows()==vector2.nrows(), f"Matrices must have the same number of rows"
    
    assert vector1.base_ring()==ZZ or vector1.base_ring()==QQ, f"ScalarProduct wants integers or matrix as input"
    assert vector2.base_ring()==ZZ or vector2.base_ring()==QQ, f"ScalarProduct wants integers or matrix as input"

    return (vector1.T*vector2)[0,0]

In [32]:
def NormSquared(vector):
    ## returns the squared norm of a (vertical) vector, aka n times 1 matrix
    
    return ScalarProduct(vector,vector)

In [33]:
def f_lower(x):
    ## returns a non-negative rational value, that is smaller (or equal) to the square root of the input
    ## THE LAST 3 LINES OF THE FUNCTION SUFFICE TO SHOW IT'S FUNCTIONALITY
    
    ## We want to have a very small relative error and not have
    ## the numerators and denominators get unnecessarily large.
    ## Also, I want the denominator to be of the form 10^n
    
    assert isinstance(x, Rational) or isinstance(x, Integer), f"Give me rational inputs, please! You gave me {type(x)}"
    assert x>=0, f"I really tried to find the square root of {x}, but I couldn't"

    if x==0:
        return 0
    
    
    #####################################################
    ## first we find the unique(!) integer a such that ##
    ## x*100^a in [10^20,10^22)                        ##
    #####################################################

    ## initial guess for a:
    a=-floor(log(float(x))/log(100.))+10
  
    while x*100^a < 10^20 or x*100^a >= 10^22: ## repeat until x*100^a in [10^20,10^22)
        if x*100^a < 10^20:
            a+=1
        if x*100^a >= 10^22:
            a-=1

    ## we now have x*100^a in [10^20,10^22)

    ##############################################
    ## we find the unique(!) positive integer b ##
    ## such that b^2<= x*100^a < (b+1)^2        ##
    ##############################################

      
    ## initial guess for b:
    b=floor(sqrt(float(x*100^a)))
  
    while b^2> x*100^a or x*100^a >= (b+1)^2:
        if b^2 > x*100^a:
            b-=1
        if (b+1)^2 <= x*100^a:
            b+=1
            
    ############################
    ## we are ready to return ##
    ############################

    ## as b²<= x*100^a, we have b/10^a<=sqrt(x)

    result=b/10^a

    ## a few 'asserts' to ensure that everything works properly: 
    assert result>=0, f"The approximation should be non-negative"
    assert result^2<=x, f"We got {result} as a lower approximation of the sqrt of {x}"
    assert isinstance(result, Rational) or isinstance(result, Integer), f"Something went wrong!"
    
    return result

In [34]:
def f_upper(x):
    ## returns a non-negative rational value, that is greater (or equal) to the square root of the input
    
    ## We want to have a very small relative error and not have
    ## the numerators and denominators get unnecessarily large.

    assert isinstance(x, Rational) or isinstance(x, Integer), f"Give me rational inputs, please! You gave me {type(x)}"
    assert x>=0, f"I really tried to find the square root of {x}, but I couldn't"

    if x==0:
        return 0
    
    result=x/f_lower(x)

    assert result>=0, f"The approximation should be non-negative"
    assert result^2>=x, f"We got {result} as an upper approximation of the sqrt of {x}"
    assert isinstance(result, Rational) or isinstance(result, Integer), f"Something went wrong!"
    
    return result

**Example Code**: Let's approximate the sqaure-root of 3.141

In [35]:
print(f"The sqaure-root of 3.141 is {sqrt(3.141)}...")
print(f"Its lower approximation is  {f_lower(3141/1000)+0.0}... = {f_lower(3141/1000)}")
print(f"Its upper approximation is  {f_upper(3141/1000)+0.0}... = {f_upper(3141/1000)}")

The sqaure-root of 3.141 is 1.77228665852903...
Its lower approximation is  1.77228665850000... = 3544573317/2000000000
Its upper approximation is  1.77228665855806... = 2094000000/1181524439


# 3 Verifying the solution <a id="3"></a>

Finally, we are ready to verify that the solution-tree indeed proves the non-existance of solutions to Rupert's Property of the sexygon. This is done in the 3 following steps:
- [3.1 General solution-tree integrety](#3.1)
- [3.2 Weak Theorems](#3.2)
- [3.3 Strong Theorem](#3.3)

## 3.1 General solution-tree integrety <a id="3.1"></a>

Here we test, that the solution tree is indeed of the desired form. We check:
1. [The Root corresponds to the correct subset of R⁵](#Test1)
2. [IDs are correct](#Test2), i.e. numbered sequentially
3. [The nodetypes are correct](#Test3), i.e. each node is marked either as weak theorem, strong theorem of inner node
4. [Child nodes exist and correspond to the correct intervals](#Test4), i.e. for each node that is marked as inner node, there are its children and the union of their intervals correspond to the interval of the inital node 

**Test 1**: The root corresponds to the correct subset of R⁵ <a id="Test1"></a>

In [17]:
row=TREE.loc[int(0)]
assert row["T1_min"]==INTERVAL_DENOMINATOR*0
assert row["T1_max"]==INTERVAL_DENOMINATOR*42/100
assert row["V1_min"]==INTERVAL_DENOMINATOR*0
assert row["V1_max"]==INTERVAL_DENOMINATOR*315/100
assert row["T2_min"]==INTERVAL_DENOMINATOR*0
assert row["T2_max"]==INTERVAL_DENOMINATOR*42/100
assert row["V2_min"]==INTERVAL_DENOMINATOR*0
assert row["V2_max"]==INTERVAL_DENOMINATOR*158/100
assert row["A_min"] ==INTERVAL_DENOMINATOR*(-158/100)
assert row["A_max"] ==INTERVAL_DENOMINATOR*158/100
print(f"Test 1 passed! The root represents the correct interval:")
print(f"    The root represents the interval theta1 = [{row["T1_min"]/INTERVAL_DENOMINATOR} , {row["T1_max"]/INTERVAL_DENOMINATOR}]")
print(f"    The root represents the interval phi1   = [{row["V1_min"]/INTERVAL_DENOMINATOR} , {row["V1_max"]/INTERVAL_DENOMINATOR}]")
print(f"    The root represents the interval theta2 = [{row["T2_min"]/INTERVAL_DENOMINATOR} , {row["T2_max"]/INTERVAL_DENOMINATOR}]")
print(f"    The root represents the interval phi2   = [{row["V2_min"]/INTERVAL_DENOMINATOR} , {row["V2_max"]/INTERVAL_DENOMINATOR}]")
print(f"    The root represents the interval alpha  = [{row["A_min"] /INTERVAL_DENOMINATOR} , {row["A_max"] /INTERVAL_DENOMINATOR}]")


Test 1 passed! The root represents the correct interval:
    The root represents the interval theta1 = [0.0 , 0.42]
    The root represents the interval phi1   = [0.0 , 3.15]
    The root represents the interval theta2 = [0.0 , 0.42]
    The root represents the interval phi2   = [0.0 , 1.58]
    The root represents the interval alpha  = [-1.58 , 1.58]


**Test 2**: IDs are correct <a id="Test2"></a>

In [18]:
## takes some time to start up :)
column_ID = vector(TREE["ID"].tolist())
for i in range(TREE.shape[0]):
    assert column_ID[i]==i+1
    if i%10000==0:
        print("Test 2: ",i,"of",TREE.shape[0],end="\r")
print("Test 2 passed! The ID's of the nodes are numbered sequentially")

Test 2 passed! The ID's of the nodes are numbered sequentially


**Test 3**: Nodetypes are correct <a id="Test3"></a>

In [19]:
nrWeak       = (TREE["nodetype"] == NODETYPE_WEAK).sum()
nrStrong     = (TREE["nodetype"] == NODETYPE_STRONG).sum()
nrInnerNodes = (TREE["nodetype"] == NODETYPE_INNER_NODE).sum()

assert nrWeak+nrStrong+nrInnerNodes==TREE.shape[0]

print("Test 3 passed! There are only the 3 expected nodetypes:")
print(f"    There are {nrWeak} nodes with nodetype {NODETYPE_WEAK}, i.e. weak Theorems")
print(f"    There are {nrStrong} nodes with nodetype {NODETYPE_STRONG}, i.e. strong Theorems")
print(f"    There are {nrInnerNodes} nodes with nodetype {NODETYPE_INNER_NODE}, i.e. split into subintervals")

Test 3 passed! There are only the 3 expected nodetypes:
    There are 17492530 nodes with nodetype 1, i.e. weak Theorems
    There are 622715 nodes with nodetype 2, i.e. strong Theorems
    There are 585200 nodes with nodetype 3, i.e. split into subintervals


**Test 4**: All children of inner nodes exist as expected <a id="Test4"></a>

In [20]:
startTime=time.time()

for i in range(TREE.shape[0]):
    if i%3000==0:
        print(f"Processing: {i} of {TREE.shape[0]}; remaining: {remainingTime(i,TREE.shape[0],startTime)}", end='\r')
    
    nodetype=TREE["nodetype"].iloc[int(i)]

    if nodetype!=NODETYPE_INNER_NODE: ## only look at inner nodes
        continue
    
    nrChildren=TREE["nrChildren"].iloc[int(i)]
    split=TREE["split"].iloc[int(i)]
    IDfirstChild=TREE["IDfirstChild"].iloc[int(i)]
    
    T1_min=TREE["T1_min"].iloc[int(i)]
    T1_max=TREE["T1_max"].iloc[int(i)]
    V1_min=TREE["V1_min"].iloc[int(i)]
    V1_max=TREE["V1_max"].iloc[int(i)]
    T2_min=TREE["T2_min"].iloc[int(i)]
    T2_max=TREE["T2_max"].iloc[int(i)]
    V2_min=TREE["V2_min"].iloc[int(i)]
    V2_max=TREE["V2_max"].iloc[int(i)]
    A_min =TREE["A_min"].iloc[int(i)]
    A_max =TREE["A_max"].iloc[int(i)]
    
    
    
    if split<=5: 
        ## only one of the intervals will be split
        ## split==1 -> only theta1 will be split
        ## split==2 -> only phi1   will be split
        ## split==3 -> only theta2 will be split
        ## split==4 -> only phi2   will be split
        ## split==5 -> only alpha  will be split

        for childNr in range(nrChildren):
            childID=IDfirstChild+childNr-1
            
            assert split==1 or TREE["T1_min"].iloc[int(childID)]==T1_min
            assert split==1 or TREE["T1_max"].iloc[int(childID)]==T1_max
            assert split==2 or TREE["V1_min"].iloc[int(childID)]==V1_min
            assert split==2 or TREE["V1_max"].iloc[int(childID)]==V1_max
            assert split==3 or TREE["T2_min"].iloc[int(childID)]==T2_min
            assert split==3 or TREE["T2_max"].iloc[int(childID)]==T2_max
            assert split==4 or TREE["V2_min"].iloc[int(childID)]==V2_min
            assert split==4 or TREE["V2_max"].iloc[int(childID)]==V2_max
            assert split==5 or TREE["A_min"].iloc[int(childID)] ==A_min
            assert split==5 or TREE["A_max"].iloc[int(childID)] ==A_max

            if split==1:
                subIntervalLength=(T1_max-T1_min)/nrChildren
                assert TREE["T1_min"].iloc[int(childID)]==T1_min+subIntervalLength*childNr
                assert TREE["T1_max"].iloc[int(childID)]==T1_min+subIntervalLength*childNr+subIntervalLength
            if split==2:
                subIntervalLength=(V1_max-V1_min)/nrChildren
                assert TREE["V1_min"].iloc[int(childID)]==V1_min+subIntervalLength*childNr
                assert TREE["V1_max"].iloc[int(childID)]==V1_min+subIntervalLength*childNr+subIntervalLength
            if split==3:
                subIntervalLength=(T2_max-T2_min)/nrChildren
                assert TREE["T2_min"].iloc[int(childID)]==T2_min+subIntervalLength*childNr
                assert TREE["T2_max"].iloc[int(childID)]==T2_min+subIntervalLength*childNr+subIntervalLength
            if split==4:
                subIntervalLength=(V2_max-V2_min)/nrChildren
                assert TREE["V2_min"].iloc[int(childID)]==V2_min+subIntervalLength*childNr
                assert TREE["V2_max"].iloc[int(childID)]==V2_min+subIntervalLength*childNr+subIntervalLength
            if split==5:
                subIntervalLength=(A_max-A_min)/nrChildren
                assert TREE["A_min"].iloc[int(childID)]==A_min+subIntervalLength*childNr
                assert TREE["A_max"].iloc[int(childID)]==A_min+subIntervalLength*childNr+subIntervalLength
                
            

    if split==6:
        ## all intervals will be split in half
        assert nrChildren==32

        ## we expect all 32 children to be stored in 32 consequtive rows of the TREE
        ## the first one should be at IDfirstChild
        ## we now iterate over all subintervals of the children, and verify their position in the solution tree
        
        childNr=0
        for a1 in range(2):
            for a2 in range(2):
                for a3 in range(2):
                    for a4 in range(2):
                        for a5 in range(2):
                            
                            childID=IDfirstChild+childNr-1 ## this -1 annoys me
                            childNr+=1 
                            

                            if a1==0:
                                assert TREE["T1_min"].iloc[int(childID)]==T1_min
                                assert TREE["T1_max"].iloc[int(childID)]==(T1_min+T1_max)/2
                            else:
                                assert TREE["T1_min"].iloc[int(childID)]==(T1_min+T1_max)/2
                                assert TREE["T1_max"].iloc[int(childID)]==T1_max

                            if a2==0:
                                assert TREE["V1_min"].iloc[int(childID)]==V1_min
                                assert TREE["V1_max"].iloc[int(childID)]==(V1_min+V1_max)/2
                            else:
                                assert TREE["V1_min"].iloc[int(childID)]==(V1_min+V1_max)/2
                                assert TREE["V1_max"].iloc[int(childID)]==V1_max

                            if a3==0:
                                assert TREE["T2_min"].iloc[int(childID)]==T2_min
                                assert TREE["T2_max"].iloc[int(childID)]==(T2_min+T2_max)/2
                            else:
                                assert TREE["T2_min"].iloc[int(childID)]==(T2_min+T2_max)/2
                                assert TREE["T2_max"].iloc[int(childID)]==T2_max

                            if a4==0:
                                assert TREE["V2_min"].iloc[int(childID)]==V2_min
                                assert TREE["V2_max"].iloc[int(childID)]==(V2_min+V2_max)/2
                            else:
                                assert TREE["V2_min"].iloc[int(childID)]==(V2_min+V2_max)/2
                                assert TREE["V2_max"].iloc[int(childID)]==V2_max

                            if a5==0:
                                assert TREE["A_min"].iloc[int(childID)]==A_min
                                assert TREE["A_max"].iloc[int(childID)]==(A_min+A_max)/2
                            else:
                                assert TREE["A_min"].iloc[int(childID)]==(A_min+A_max)/2
                                assert TREE["A_max"].iloc[int(childID)]==A_max
            


KeyboardInterrupt: 

In [21]:
## maybe useful code
TREE['split'].value_counts()

split
6    577395
5      7200
4       480
3       120
2         4
1         1
Name: count, dtype: Int64

## 3.2 Weak Theorems <a id="3.2"></a>

In [36]:
startTime=time.time()

for row in TREE.itertuples():
    if row.ID%1000==0:
        print(f"Processing: {row.ID} of {TREE.shape[0]}; remaining: {remainingTime(row.ID,TREE.shape[0],startTime)}", end='\r')
    
    if row.nodetype!= NODETYPE_WEAK:
        continue
 
    wx = Integer(row.wx_nominator)
    wy = Integer(row.wy_nominator)
    den = Integer(row.w_denominator)

    w=matrix([[wx/den],[wy/den]])

    assert w.T*w==matrix([1]), f"The vector w no unit vector, ID={row.ID} w={w}"


    T1_min=Integer(row.T1_min)/INTERVAL_DENOMINATOR
    T1_max=Integer(row.T1_max)/INTERVAL_DENOMINATOR
    V1_min=Integer(row.V1_min)/INTERVAL_DENOMINATOR
    V1_max=Integer(row.V1_max)/INTERVAL_DENOMINATOR
    T2_min=Integer(row.T2_min)/INTERVAL_DENOMINATOR
    T2_max=Integer(row.T2_max)/INTERVAL_DENOMINATOR
    V2_min=Integer(row.V2_min)/INTERVAL_DENOMINATOR
    V2_max=Integer(row.V2_max)/INTERVAL_DENOMINATOR
    A_min =Integer(row.A_min )/INTERVAL_DENOMINATOR
    A_max =Integer(row.A_max )/INTERVAL_DENOMINATOR

    t1=(T1_min+T1_max)/2
    v1=(V1_min+V1_max)/2
    t2=(T2_min+T2_max)/2
    v2=(V2_min+V2_max)/2
    a=(A_min+A_max)/2

    eps=max(T1_max-T1_min,V1_max-V1_min,T2_max-T2_min,V2_max-V2_min,A_max-A_min)/2
  
    S_=polyRat[row.S_index-1,:].T ## S-tilde or something. The "-1" is needed because changing from R to Sage
    
     
    G_=((R(a)*M(t1,v1)*S_).T*w-
    eps*abs_matrix((R_alpha_prime(a)*M(t1,v1)*S_).T*w)-
    eps*abs_matrix((R(a)*M_theta_prime(t1,v1)*S_).T*w)-
    eps*abs_matrix((R(a)*M_phi_prime(t1,v1)*S_).T*w)-
    9/2*eps^2-
    4*kappa*2-
    12*kappa*2*eps)
  
    assert G_.dimensions()==(1,1), f"Something with G_ went terribly wrong"
    G_=G_[0,0]

    
    Term1=polyRat*M(t2,v2).T*w
    Term2=polyRat*M_theta_prime(t2,v2).T*w
    Term3=polyRat*M_phi_prime(t2,v2).T*w
    
    HP=Term1+eps*abs_matrix(Term2)+eps*abs_matrix(Term3)
    a
    H_=max(HP)[0]+2*eps^2+3*kappa*2+6*kappa*2*eps
    
    assert G_>H_, f"So... it seems the weak Theorem cannot be applied"

KeyboardInterrupt: 

In [44]:
max(HP)[0]

111248956949155980510006866836271825413418781922651251891823422083291769580633448463886656174526003894744186858947131071964830992464774510798655090530243602079505162739730106489216305041431727829034769629440717063347216930797305425379302095863/115198985340477794098018048277083084444265772357792708009481737835686896735238835352371200000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

## 3.3 Strong Theorem  <a id="3.3"></a>

There are many statements that need to be verified to apply the strong theorem. Here we focus on that 'L-condition'. This condition is special, as it only depends on which 6 points of the polyhedron are chosen (P1,P2,P3,Q1,Q2,Q3) and no other parameters. Hence, we can verify this condition for all strong theorem, by only considering which unique sextuples of points are used.

### 3.3.1 Verifying strong - only that L condition

First lets filter out only the strong nodes and consider their sextupel:

In [37]:
#There exists an orthonormal matrix $L$ (i.e. $LL^T=Id$) such that $P_i=LQ_i$ for $i=1,2,3$.  
TREE_filtered = TREE[['P1_index', 'P2_index', 'P3_index', 'Q1_index', 'Q2_index', 'Q3_index']][TREE['nodetype'] == NODETYPE_STRONG]
TREE_filtered

,P1_index,P2_index,P3_index,Q1_index,Q2_index,Q3_index
245,31,32,39,80,81,88
248,31,32,39,80,81,88
249,31,32,39,80,81,88
251,31,32,39,80,81,88
257,31,32,39,80,81,88
...,...,...,...,...,...,...
18633110,76,84,77,34,41,33
18640764,76,87,80,83,87,79
18640776,76,87,80,83,87,79
18640779,76,87,80,83,87,79


Now, we can focus only on the UNIQUE sextuples:

In [38]:
TREE_filtered=TREE_filtered.drop_duplicates()
TREE_filtered

,P1_index,P2_index,P3_index,Q1_index,Q2_index,Q3_index
245,31,32,39,80,81,88
1348,31,32,39,42,43,35
3957,31,32,39,43,44,36
8661,31,32,39,81,82,89
12309,31,32,39,44,45,37
...,...,...,...,...,...,...
17902083,31,78,87,38,81,87
17908476,45,79,88,39,80,86
18569592,76,85,79,83,89,80
18622726,82,90,83,42,34,41


These 577 sextuples can be checked by the following code, that uses one of the results of the paper:

In [39]:
ID = matrix(SR, 3,3)
ID[0,0]=1
ID[1,1]=1
ID[2,2]=1

startTime=time.time()

for rowNr, row in enumerate(TREE_filtered.itertuples(), start=1):
    print(f"Processing: {rowNr} of {TREE_filtered.shape[0]}; remaining: {remainingTime(rowNr,TREE_filtered.shape[0],startTime)}", end='\r')
    
    P1_index=row.P1_index-1
    P2_index=row.P2_index-1
    P3_index=row.P3_index-1
    Q1_index=row.Q1_index-1
    Q2_index=row.Q2_index-1
    Q3_index=row.Q3_index-1
  
    P1=poly[P1_index,:].T
    P2=poly[P2_index,:].T
    P3=poly[P3_index,:].T
    Q1=poly[Q1_index,:].T
    Q2=poly[Q2_index,:].T
    Q3=poly[Q3_index,:].T
    
    
    assert P1.T*P1==Q1.T*Q1
    assert P2.T*P2==Q2.T*Q2
    assert P3.T*P3==Q3.T*Q3
    
    assert P1.T*P2==Q1.T*Q2
    assert P1.T*P3==Q1.T*Q3
    assert P2.T*P3==Q2.T*Q3

    Q_matrix = matrix(SR, 3,3)
    Q_matrix[:,0]=Q1
    Q_matrix[:,1]=Q2
    Q_matrix[:,2]=Q3

    assert rank(Q_matrix)==3

KeyboardInterrupt: 

### 3.3.2 Verifying strong - took like 15 hours

In [40]:
startTime=time.time()
nrStrong = (TREE["nodetype"] == NODETYPE_STRONG).sum()
nrVerified=0

for row in TREE.itertuples():
    if row.nodetype!= NODETYPE_STRONG:
        continue

    
    nrVerified+=1
    if nrVerified%20==0:
        print(f"Processing: strong Theorem {nrVerified} of {nrStrong}; remaining: {remainingTime(nrVerified,nrStrong,startTime)}", end='\r')
    
    
    
    T1_min=Integer(row.T1_min)/INTERVAL_DENOMINATOR
    T1_max=Integer(row.T1_max)/INTERVAL_DENOMINATOR
    V1_min=Integer(row.V1_min)/INTERVAL_DENOMINATOR
    V1_max=Integer(row.V1_max)/INTERVAL_DENOMINATOR
    T2_min=Integer(row.T2_min)/INTERVAL_DENOMINATOR
    T2_max=Integer(row.T2_max)/INTERVAL_DENOMINATOR
    V2_min=Integer(row.V2_min)/INTERVAL_DENOMINATOR
    V2_max=Integer(row.V2_max)/INTERVAL_DENOMINATOR
    A_min =Integer(row.A_min )/INTERVAL_DENOMINATOR
    A_max =Integer(row.A_max )/INTERVAL_DENOMINATOR

    t1=(T1_min+T1_max)/2
    v1=(V1_min+V1_max)/2
    t2=(T2_min+T2_max)/2
    v2=(V2_min+V2_max)/2
    a=(A_min+A_max)/2

    eps=max(T1_max-T1_min,V1_max-V1_min,T2_max-T2_min,V2_max-V2_min,A_max-A_min)/2
  
    P1_index=row.P1_index-1
    P2_index=row.P2_index-1
    P3_index=row.P3_index-1
    Q1_index=row.Q1_index-1
    Q2_index=row.Q2_index-1
    Q3_index=row.Q3_index-1
  
    P1=polyRat[P1_index,:].T
    P2=polyRat[P2_index,:].T
    P3=polyRat[P3_index,:].T
    Q1=polyRat[Q1_index,:].T
    Q2=polyRat[Q2_index,:].T
    Q3=polyRat[Q3_index,:].T
    
    r=Integer(row.r)/1000 ## SOLLTE KONSTANTE SEIN
    s_p=1
    s_q=Integer(row.s_q)
    assert s_q==1 or s_q==-1, f"s_q is neither 1 or -1"
  
    
    R_pi_2=matrix(QQ,[[0,-1],[1,0]]) ## is set to R(pi/2)
    
    X_1=X(t1,v1)
    X_2=X(t2,v2)
    
    M_t1_v1=M(t1,v1)
    M_t2_v2=M(t2,v2)

    r1=(R(a)*M_t1_v1*P1 -M_t2_v2*Q1)/2
    r2=(R(a)*M_t1_v1*P2 -M_t2_v2*Q2)/2
    r3=(R(a)*M_t1_v1*P3 -M_t2_v2*Q3)/2
    
    delta=4*kappa*2+max(f_upper(NormSquared(r1)),
                        f_upper(NormSquared(r2)),
                        f_upper(NormSquared(r3)))
    
    g=283/100*eps+2*eps^2
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### That s_p condition ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~

    assert s_p*ScalarProduct(X_1,P1)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert s_p*ScalarProduct(X_1,P2)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert s_p*ScalarProduct(X_1,P3)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
  
    #~~~~~~~~~~~~~~~~~~~~~~~~~~  
    ### That s_q condition ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~  
  
    assert s_q*ScalarProduct(X_2,Q1)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert s_q*ScalarProduct(X_2,Q2)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert s_q*ScalarProduct(X_2,Q3)>142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
    ### Those many inequalities ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    assert ScalarProduct(R_pi_2*M_t1_v1*P1,M_t1_v1*P2)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"
    assert ScalarProduct(R_pi_2*M_t1_v1*P2,M_t1_v1*P3)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"
    assert ScalarProduct(R_pi_2*M_t1_v1*P3,M_t1_v1*P1)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"
    
    assert ScalarProduct(R_pi_2*M_t2_v2*Q1,M_t2_v2*Q2)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"
    assert ScalarProduct(R_pi_2*M_t2_v2*Q2,M_t2_v2*Q3)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"
    assert ScalarProduct(R_pi_2*M_t2_v2*Q3,M_t2_v2*Q1)>g+6*kappa*2^2, f"So... it seems the strong Theorem cannot be applied"

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### Points are far from the origin ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    assert f_lower(NormSquared(M_t2_v2*Q1))>= r+142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert f_lower(NormSquared(M_t2_v2*Q2))>= r+142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    assert f_lower(NormSquared(M_t2_v2*Q3))>= r+142/100*eps+3*kappa*2, f"So... it seems the strong Theorem cannot be applied"
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## That rational inequality ###
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    for i in range(1,4):
        if i==1:
            Qi_Index=Q1_index
            Qi=Q1
        if i==2:
            Qi_Index=Q2_index
            Qi=Q2
        if i==3:
            Qi_Index=Q3_index
            Qi=Q3
    
        denom1=f_upper(NormSquared(M_t2_v2*Qi))+142/100*eps+3*kappa*2
    
    
        for A_index in range(polyRat.nrows()):
            if A_index==Qi_Index:
                continue
                
            A=polyRat[A_index,:].T

            nom1=ScalarProduct(M_t2_v2*Qi,M_t2_v2*Qi-M_t2_v2*A)
            nom2=9*kappa*2^2+(f_upper(NormSquared(Qi-A))+2*kappa)*(284/100*eps+2*eps^2)
            
            nom=nom1-nom2

            
            denom2=f_upper(NormSquared(M_t2_v2*Qi-M_t2_v2*A))+284/100*eps+6*kappa*2
      
            denom=denom1*denom2
      
            frac=nom/denom
      
            assert frac>=(448/100*eps+2*delta)/(2*r), f"So... it seems the strong Theorem cannot be applied"
            

KeyboardInterrupt: 

In [41]:
def Norm_upper(x):
    return f_upper(Scalar(x,x))


SyntaxError: expected '(' (1551364444.py, line 1)